In [1]:
from fcgb.chatbots.chatbot import BaseChatBot
from fcgb.types.tools import ToolOutput
from langgraph.graph import MessagesState
from langchain_core.runnables.config import RunnableConfig
from langgraph.graph import StateGraph, START, END
from typing import List, Dict, Annotated
from pydantic import BaseModel

from operator import add
import uuid
from langchain_core.messages import ToolMessage
from langgraph.constants import Send
from fcgb.types.utils import append_or_clear, MessagesType
from fcgb.cfg.utils import model2string, dict2string

In [41]:
from fcgb.fake_models import FakeStructuredOutput
from fcgb.types.tools import JobModel

job_model = FakeStructuredOutput(JobModel)()
job_model, type(job_model)

(JobModel(job='Fake string ebxdn', restrictions='Fake string lndml', output_format='Fake string vxpwe', variables=['restrictions', 'restrictions', 'knowledge_base']),
 fcgb.types.tools.JobModel)

In [ ]:

class PlannedIterativeTaskSolver(BaseChatBot):
    def __init__(self,
                 llm,
                 job_handler,
                 initial_messages_spec,
                 internal_messages_spec,
                 memory=None,
                 init_values={},
                 prompt_manager_spec={},
                 global_inputs={}):
        super().__init__(
            llm=llm,
            initial_messages_spec=initial_messages_spec,
            internal_messages_spec=internal_messages_spec,
            memory=memory,
            init_values=init_values,
            prompt_manager_spec=prompt_manager_spec,
            global_inputs=global_inputs,
            compile=False
        )

        self.job_handler = job_handler
        self.compile_graph()

    def _set_state_class(self):
        
        class State(BaseModel):
            template_inputs: Dict[str, str]
            all_messages: MessagesType
            current_messages: MessagesType
            all_jobs: Annotated[List, append_or_clear]
            current_jobs: List = []
            current_jobs_outputs: Annotated[List[str], append_or_clear]
            knowledge_base: str
            turns: int
            thread_id: str = None
            output: str = None # type: ignore

        self.state_class = State

    @staticmethod
    def _get_jobs_list(jobs_descriptions: List) -> str:
        return '\n'.join(f'Job {i+1}: {job.job}' for i, job in enumerate(jobs_descriptions))

    def _set_planning_func(self):

        def planning(state: self.state_class): # type: ignore
            turns = state.turns + 1
            print(f'Planning turn {turns}...')
            all_jobs = self._get_jobs_list(state.all_jobs)
            template_inputs = state.template_inputs | {'turns': turns, 'knowledge_base': state.knowledge_base, 'all_jobs': all_jobs}

            system_msg, _ = self._get_internal_message('system', template_inputs)
            query_msg, _ = self._get_internal_message('query', template_inputs)
            planning_msg, _ = self._get_internal_message('planning', template_inputs)

            plan_msg = self.llm.invoke([system_msg, query_msg, planning_msg])

            return {
                'current_messages': [system_msg, query_msg, planning_msg, plan_msg], 
                'turns': turns
                }
        
        return planning
    
    def _set_process_end_routing_func(self):

        def process_end_routing(state: self.state_class): # type: ignore

            if state.turns == 1:
                return 'distribution'
            elif state.turns >  self.global_inputs['max_turns']:
                return 'report'
            else:
                print(f'Process End Routing turn {state.turns}...')
                routing_msg, answer_model = self._get_internal_message('process_end_routing', state.template_inputs)

                decision = self.llm.with_structured_output(answer_model).invoke(state.current_messages + [routing_msg]).decision

                if decision == 'report':
                    return 'report'
                else:
                    return 'distribution'
                
        def max_turns_routing(state: self.state_class): # type: ignore
            return 'report' if state.turns >= self.global_inputs['max_turns'] else 'planning'
                
            
        return process_end_routing, max_turns_routing
    
    def _set_distribution_func(self):

        def distribution(state: self.state_class): # type: ignore
            print(f'Jobs Distribution turn {state.turns}...')

            template_inputs = state.template_inputs | {'turns': state.turns, 'knowledge_base': state.knowledge_base}

            distribution_msg, answer_model = self._get_internal_message('distribution', template_inputs)

            jobs = self.llm.with_structured_output(answer_model).invoke(state.current_messages + [distribution_msg])

            return {
                'current_jobs': jobs.jobs
            }
        
        return distribution
    
    def _set_job_routing_func(self):

        def job_routing(state: self.state_class): # type: ignore

            return [Send('executor',  {'state': state, 'job': job}) for job in state.current_jobs]
        
        return job_routing
    
    def _set_executor_func(self):

        def executor(input: Dict): # type: ignore
            job = input['job']
            state = input['state']

            print(f'Job Execution turn {state.turns}: {job.job}...')

            job_dict = job.model_dump()
            variables = job_dict['variables']
            job_dict.pop('variables', None)  # Remove 'variables' key if it exists
            job_description = dict2string(job_dict)

            if len(variables) > 0:
                data_content = '\n\n# **additional data**:\n'
                variable_base = state.template_inputs | {'knowledge_base': state.knowledge_base}
                available_variables = variable_base.keys()
                for variable in variables:
                    if variable in available_variables:
                        data_content += f'## {variable}:\n{variable_base[variable]}\n\n'

                job_description += data_content
            
            job_output = self.job_handler.run(job_description=job_description)

            return {'current_jobs_outputs': job_output}
        
        return executor
    
    @staticmethod
    def concatenate_outputs(outputs: List[Dict]) -> str:
        return '\n-----\n'.join(output['output'] for output in outputs if 'output' in output)
    
    def _set_base_extension_func(self):

        def base_extension(state: self.state_class): # type: ignore
            print(f'Base Extension turn {state.turns}...')
            retrieved_data = self.concatenate_outputs(state.current_jobs_outputs)
            current_jobs = self._get_jobs_list(state.current_jobs)
            template_inputs = state.template_inputs | {'turns': state.turns, 'knowledge_base': state.knowledge_base, 'retrieved_data': retrieved_data, 'current_jobs': current_jobs}
            extension_msg, _ = self._get_internal_message('base_extension', template_inputs)

            extended_base = self.llm.invoke(state.current_messages + [extension_msg])
            return {
                'knowledge_base': extended_base.content,
                'current_messages': [extension_msg, extended_base],
            }
        
        return base_extension
    
    def _set_iteration_end_func(self):

        def iteration_end(state: self.state_class): # type: ignore
            messages_to_pass = state.current_messages[2:] if state.turns>1 else state.current_messages
            return {
                'all_messages': messages_to_pass,
                'current_messages': '__clear__',
                'all_jobs': state.current_jobs,
                'current_jobs': [],
                'current_jobs_outputs': '__clear__'
            }

        return iteration_end
    
    def _set_report_func(self):

        def report(state: self.state_class, config: RunnableConfig): # type: ignore
            print(f'Reporting turn {state.turns}...')
            template_inputs = state.template_inputs | {'turns': state.turns, 'knowledge_base': state.knowledge_base}

            report_msg, _ = self._get_internal_message('report', template_inputs)

            output = self.llm.invoke([report_msg])
            thread_id = config['configurable']['thread_id']

            return {
                'output': output.content,
                'thread_id': thread_id,
                'all_messages': state.current_messages,
                'current_messages': '__clear__',
            }
        
        return report
    
    def _compile_graph(self):
        
        planning_func = self._set_planning_func()
        process_end_routing_func, max_turns_routing_func = self._set_process_end_routing_func()
        distribution_func = self._set_distribution_func()
        job_routing_func = self._set_job_routing_func()
        executor_func = self._set_executor_func()
        base_extension_func = self._set_base_extension_func()
        iteration_end_func = self._set_iteration_end_func()
        report_func = self._set_report_func()

        workflow = StateGraph(self.state_class)
        workflow.add_node('planning', planning_func)
        workflow.add_node('distribution', distribution_func)
        workflow.add_node('executor', executor_func)
        workflow.add_node('base_extension', base_extension_func)
        workflow.add_node('iteration_end', iteration_end_func)
        workflow.add_node('report', report_func)

        workflow.add_edge(START, 'planning')
        workflow.add_conditional_edges('planning', process_end_routing_func, ['distribution', 'report'])
        workflow.add_conditional_edges('distribution', job_routing_func, ['executor'])
        workflow.add_edge('executor', 'base_extension')
        workflow.add_edge('base_extension', 'iteration_end')
        workflow.add_conditional_edges('iteration_end', max_turns_routing_func, ['planning', 'report'])
        workflow.add_edge('report', END)

        self.graph = workflow.compile(
            checkpointer=self.memory
        )

    def run(self, job: str, restrictions: str, output_format: str, data: str) -> ToolOutput:
        
        thread_id = uuid.uuid4().hex
        return self.graph.invoke(
            input={
                'template_inputs': {
                    'job': job,
                    'restrictions': restrictions,
                    'output_format': output_format,
                    'data': data
                    },
                'turns': 0,
                'knowledge_base': 'Empty'
                },
            config = {'configurable': {'thread_id': thread_id}},
            output_keys=['thread_id', 'output']
        )

In [3]:
inputs = {
    'job': 'List all Chain-of-Thought prompting variants',
    'restrictions': """
    1. List only real prompting techniques
    2. Do not include techniques that are just applications of other techniques
    3. Do not include techniques that are not related to CoT prompting""",
    'output_format': 'List of techniques with very short description of each technique',
    'data': 'None'
}

In [4]:
from fcgb.cfg.precompiled import get_llm, get_checkpointer
from fcgb.tools.spectools import PhantomResearcherSpecTool, JobHandlerSpecTool, JobHandler
from fcgb.cfg.chat_inputs_spec import PlannedIterativeTaskSolverConfig

## Fake LLM

In [20]:
fake_llm = get_llm(llm_model='fake')
memory = get_checkpointer(checkpointer_mode='local', mode='test')

tool_containers = [
    PhantomResearcherSpecTool(llm=fake_llm, memory=memory)
]

job_handler = JobHandlerSpecTool(
    llm=fake_llm,
    tool_containers=tool_containers,
    memory=memory,
)

In [21]:
task_solver = PlannedIterativeTaskSolver(
    llm=fake_llm,
    job_handler=job_handler,
    initial_messages_spec=PlannedIterativeTaskSolverConfig.initial_messages_spec,
    internal_messages_spec=PlannedIterativeTaskSolverConfig.internal_messages_spec,
    memory=memory,
    init_values=PlannedIterativeTaskSolverConfig.init_values,
    prompt_manager_spec=PlannedIterativeTaskSolverConfig.prompt_manager_spec,
    global_inputs=PlannedIterativeTaskSolverConfig.global_inputs
)

In [22]:
output = task_solver.run(**inputs)

# Final Report
for line in output['output'].split('\n'):
    print(line)

Fake LLM response 44


In [23]:
# Load whole task solver state
research_state = task_solver.get_state(output['thread_id'])
research_state

{'template_inputs': {'job': 'List all Chain-of-Thought prompting variants',
  'restrictions': '\n    1. List only real prompting techniques\n    2. Do not include techniques that are just applications of other techniques\n    3. Do not include techniques that are not related to CoT prompting',
  'output_format': 'List of techniques with very short description of each technique',
  'data': 'None'},
 'turns': 5,
 'knowledge_base': 'Fake LLM response 42',
 'current_messages': [],
 'current_jobs': [],
 'current_jobs_outputs': [],
 'all_messages': [SystemMessage(content="# **CONVERSATION PURPOSE**\n\nThis conversation aims to:\n\n1.  **Acquire Data**: Systematically gather all data relevant to the research's **MAIN TASK**.\n2.  **Plan Strategically**: Conduct iterative research, moving from general to specific topics.\n3.  **Ensure Comprehensive Output**: Continuously analyze the **MAIN TASK** description to ensure all necessary data is collected.\n\n---\n\n# **WORKFLOW OVERVIEW**\n\n## **W

In [9]:
# All messages from the process
for msg in research_state['all_messages']:
    msg.pretty_print()

================================ System Message ================================
Name: system

# **CONVERSATION PURPOSE**

This conversation aims to:

1.  **Acquire Data**: Systematically gather all data relevant to the research's **MAIN TASK**.
2.  **Plan Strategically**: Conduct iterative research, moving from general to specific topics.
3.  **Ensure Comprehensive Output**: Continuously analyze the **MAIN TASK** description to ensure all necessary data is collected.

---

# **WORKFLOW OVERVIEW**

## **Workflow nodes**
Each node has a specific role. Focus solely on the current node's task without anticipating subsequent steps.
- Task Specification: The user provides the main task description and all necessary details.
- Analysis: Assess the current knowledge base and formulate further research directions.
- Job Distribution: Based on the analysis, delegate specific research jobs to specialized agents.
- Report Extension: Integrate collected results from agents to expand the knowledge 

In [66]:
# All delegated jobs
for i,job in enumerate(research_state['all_jobs']):
    print(f'Job {i+1}:')
    print(job.job)
    print(job.restrictions)
    print(job.output_format)
    print(job.variables)
    print('-'*50)


Job 1:
Fake string pzixp
Fake string xudic
Fake string vvegy
['knowledge_base', 'restrictions']
--------------------------------------------------
Job 2:
Fake string rxiod
Fake string reelv
Fake string stjdj
['knowledge_base']
--------------------------------------------------
Job 3:
Fake string omsdr
Fake string klkst
Fake string lletu
['knowledge_base']
--------------------------------------------------
Job 4:
Fake string kvtoh
Fake string ahehe
Fake string nkznr
['restrictions']
--------------------------------------------------
Job 5:
Fake string abixy
Fake string obprb
Fake string bjhty
['knowledge_base', 'restrictions', 'restrictions']
--------------------------------------------------
Job 6:
Fake string ltbon
Fake string vobqr
Fake string qbhay
['restrictions', 'knowledge_base', 'knowledge_base', 'knowledge_base']
--------------------------------------------------
Job 7:
Fake string tukhl
Fake string xpsty
Fake string szfit
['restrictions', 'knowledge_base', 'knowledge_base', 'k

## Real LLM

In [5]:
llm = get_llm(llm_model='google')
memory = get_checkpointer(checkpointer_mode='local', mode='test')

tool_containers = [
    PhantomResearcherSpecTool(llm=llm, memory=memory)
]

job_handler = JobHandlerSpecTool(
    llm=llm,
    tool_containers=tool_containers,
    memory=memory,
)

In [6]:
task_solver = PlannedIterativeTaskSolver(
    llm=llm,
    job_handler=job_handler,
    initial_messages_spec=PlannedIterativeTaskSolverConfig.initial_messages_spec,
    internal_messages_spec=PlannedIterativeTaskSolverConfig.internal_messages_spec,
    memory=memory,
    init_values=PlannedIterativeTaskSolverConfig.init_values,
    prompt_manager_spec=PlannedIterativeTaskSolverConfig.prompt_manager_spec,
    global_inputs=PlannedIterativeTaskSolverConfig.global_inputs
)

In [7]:
output = task_solver.run(**inputs)

# Final Report
for line in output['output'].split('\n'):
    print(line)

*   **Zero-shot CoT:** Elicits reasoning using "Let's think step by step" without requiring any examples.
*   **Self-Consistency:** Generates multiple reasoning paths and selects the most consistent answer by a majority vote.
*   **Least-to-Most Prompting:** Decomposes problems into simpler subproblems, solved sequentially to provide context.
*   **Plan-and-Solve Prompting:** Separates planning from execution by prompting the model to first create a plan and then execute it.
*   **Graph of Thoughts (GoT):** Generalizes chain-of-thought to a graph structure, enabling exploration of multiple paths and backtracking.
*   **Tree of Thoughts (ToT):** Explores multiple reasoning paths in parallel, allowing backtracking and evaluation of different thought sequences.
*   **Chain-of-Verification (CoV):** Augments CoT by incorporating verification steps to improve the reliability of the reasoning process.
*   **CoT with Memory (CoTM):** Augments CoT with a memory component to store past reasoning

In [8]:
# Load whole task solver state
research_state = task_solver.get_state(output['thread_id'])
#research_state

In [9]:
# All messages from the process
for msg in research_state['all_messages']:
    msg.pretty_print()

================================ System Message ================================
Name: system

You are designated to perform a complex task that requires you to solve a problem by breaking it down into smaller, manageable steps. 
Your goal is to manage whole realization of the task including detailed planning, specific jobs delegation and writing the final report.

Pay special attention to the first received message, as it contains the task description and all necessary details.
Especially when writing the final report, ensure that it fits into the provided output format.

You will work in an iterative way with following schema:
1. **Task Specification**: The user will provide a task description, including all necessary details.
2. **Planning**: With input from previous steps you will describe your reasoning and plan next step or call the end of the process.
3. **Job Distribution**: Using the plan, you will delegate specific jobs that will be performed by specialized agents. Be careful

In [37]:
# Knowledge Base
for line in research_state['knowledge_base'].split('\n'):
    print(line)

```
## Chain-of-Thought Prompting Techniques

### Basic CoT Techniques
1.  **Standard Chain-of-Thought Prompting:**
    *   **Description:** Prompting the model to generate intermediate reasoning steps before the final answer, demonstrated through example question-answer pairs.
2.  **Zero-Shot Chain-of-Thought Prompting:**
    *   **Description:** Prompting the model to think step-by-step by adding "Let's think step by step" to the question, without providing examples.

### Advanced CoT Techniques
1.  **Least-to-Most Prompting:**
    *   **Description:** Breaking down a complex problem into simpler subproblems solved sequentially, ordered by difficulty/dependency.
2.  **Tree of Thoughts (ToT):**
    *   **Description:** Exploring multiple reasoning paths at each step, maintaining a tree structure. Uses search algorithms to navigate the tree.
3.  **Graph of Thoughts (GoT):**
    *   **Description:** Structuring the reasoning process as a graph, where nodes represent thoughts and edges r

In [10]:
# All delegated jobs
for i,job in enumerate(research_state['all_jobs']):
    print(f'Job {i+1}:')
    print('Job: ', job.job)
    print('Restrictions: ', job.restrictions)
    print('Output Format: ', job.output_format)
    print('Variables: ', job.variables)
    print('-'*50)


Job 1:
Job:  Conduct a literature review to identify different Chain-of-Thought (CoT) prompting variants. Focus on academic papers, blog posts, and articles that discuss modifications or extensions to the standard CoT prompting technique.  Identify the core idea behind each variant and how it differs from the standard CoT.
Restrictions:  List only real prompting techniques. Do not include techniques that are just applications of other techniques. Do not include techniques that are not related to CoT prompting.
Output Format:  A list of potential CoT variants with concise descriptions (max 50 words each) focusing on the core idea and differences from standard CoT, and links to the source materials.
Variables:  []
--------------------------------------------------
Job 2:
Job:  Perform a web search to identify different Chain-of-Thought (CoT) prompting variants. Focus on practical examples, tutorials, and discussions in online communities (e.g., Stack Overflow, Reddit) related to CoT prom

## Partial execution

In [5]:
llm = get_llm(llm_model='google')
memory = get_checkpointer(checkpointer_mode='local', mode='test')

tool_containers = [
    PhantomResearcherSpecTool(llm=llm, memory=memory)
]

job_handler = JobHandlerSpecTool(
    llm=llm,
    tool_containers=tool_containers,
    memory=memory,
)

In [6]:
task_solver = PlannedIterativeTaskSolver(
    llm=llm,
    job_handler=job_handler,
    initial_messages_spec=PlannedIterativeTaskSolverConfig.initial_messages_spec,
    internal_messages_spec=PlannedIterativeTaskSolverConfig.internal_messages_spec,
    memory=memory,
    init_values=PlannedIterativeTaskSolverConfig.init_values,
    prompt_manager_spec=PlannedIterativeTaskSolverConfig.prompt_manager_spec,
    global_inputs=PlannedIterativeTaskSolverConfig.global_inputs
)

### planning

In [26]:
thread_id = 'planning1'
research_state = task_solver.graph.invoke(
            input={
                'template_inputs': inputs,
                'turns': 0,
                'knowledge_base': 'Empty'
                },
            config = {'configurable': {'thread_id': thread_id}},
            interrupt_after='planning'
        )

In [27]:
for msg in research_state['current_messages']:
    msg.pretty_print()

================================ System Message ================================
Name: system

# **CONVERSATION PURPOSE**

This conversation aims to:

1.  **Acquire Data**: Systematically gather all data relevant to the research's **MAIN TASK**.
2.  **Plan Strategically**: Conduct iterative research, moving from general to specific topics.
3.  **Ensure Comprehensive Output**: Continuously analyze the **MAIN TASK** description to ensure all necessary data is collected.

---

# **WORKFLOW OVERVIEW**

## **Workflow nodes**
Each node has a specific role. Focus solely on the current node's task without anticipating subsequent steps.
- Task Specification: The user provides the main task description and all necessary details.
- Analysis: Assess the current knowledge base and formulate further research directions.
- Jobs Distribution: Based on the analysis, delegate specific research jobs to specialized agents.
- Jobs Execution: Agents execute the assigned jobs, gathering data and insights.
- 

In [28]:
task_solver.get_state(thread_id)

{'template_inputs': {'job': 'List all Chain-of-Thought prompting variants',
  'restrictions': '\n    1. List only real prompting techniques\n    2. Do not include techniques that are just applications of other techniques\n    3. Do not include techniques that are not related to CoT prompting',
  'output_format': 'List of techniques with very short description of each technique',
  'data': 'None'},
 'turns': 1,
 'knowledge_base': 'Empty',
 'current_messages': [SystemMessage(content="# **CONVERSATION PURPOSE**\n\nThis conversation aims to:\n\n1.  **Acquire Data**: Systematically gather all data relevant to the research's **MAIN TASK**.\n2.  **Plan Strategically**: Conduct iterative research, moving from general to specific topics.\n3.  **Ensure Comprehensive Output**: Continuously analyze the **MAIN TASK** description to ensure all necessary data is collected.\n\n---\n\n# **WORKFLOW OVERVIEW**\n\n## **Workflow nodes**\nEach node has a specific role. Focus solely on the current node's tas

### jobs distribution

In [38]:
thread_id = 'distribution1'
research_state = task_solver.graph.invoke(
            input={
                'template_inputs': inputs,
                'turns': 0,
                'knowledge_base': 'Empty'
                },
            config = {'configurable': {'thread_id': thread_id}},
            interrupt_after='distribution'
        )

Planning turn 1...
Jobs Distribution turn 1...


In [39]:
for msg in research_state['current_messages']:
    msg.pretty_print()

================================ System Message ================================
Name: system

# **CONVERSATION PURPOSE**

This conversation aims to:

1.  **Acquire Data**: Systematically gather all data relevant to the research's **MAIN TASK**.
2.  **Plan Strategically**: Conduct iterative research, moving from general to specific topics.
3.  **Ensure Comprehensive Output**: Continuously analyze the **MAIN TASK** description to ensure all necessary data is collected.

---

# **WORKFLOW OVERVIEW**

## **Workflow nodes**
Each node has a specific role. Focus solely on the current node's task without anticipating subsequent steps.
- Task Specification: The user provides the main task description and all necessary details.
- Analysis: Assess the current knowledge base and formulate further research directions.
- Jobs Distribution: Based on the analysis, delegate specific research jobs to specialized agents.
- Jobs Execution: Agents execute the assigned jobs, gathering data and insights.
- 

In [40]:
for i,job in enumerate(research_state['current_jobs']):
    print(f'Job {i+1}:')
    print(job.job)
    print(job.restrictions)
    print(job.output_format)
    print(job.variables)
    print('-'*50)

Job 1:
Research and summarize the Chain-of-Thought (CoT) prompting technique, including its core principles and variations.
Focus on the prompting technique itself, not its applications. Adhere to the general restrictions.
A short description of the technique.
['restrictions']
--------------------------------------------------
Job 2:
Investigate and summarize Self-Consistency and Least-to-Most Prompting as Chain-of-Thought variants.
Focus on the prompting techniques themselves, not their applications. Adhere to the general restrictions.
Short descriptions of each technique, highlighting their differences and similarities.
['restrictions']
--------------------------------------------------
Job 3:
Research and summarize Tree-of-Thoughts (ToT) and Graph of Thoughts (GoT) prompting techniques.
Focus on the prompting techniques themselves, not their applications. Adhere to the general restrictions.
Short descriptions of each technique, highlighting their differences and similarities.
['rest

### knowledge extension

In [7]:
thread_id = 'base_extension1'
research_state = task_solver.graph.invoke(
            input={
                'template_inputs': inputs,
                'turns': 0,
                'knowledge_base': 'Empty'
                },
            config = {'configurable': {'thread_id': thread_id}},
            interrupt_after='base_extension'
        )

Planning turn 1...
Jobs Distribution turn 1...


In [8]:
for msg in research_state['current_messages']:
    msg.pretty_print()

================================ System Message ================================
Name: system

# **CONVERSATION PURPOSE**

This conversation aims to:

1.  **Acquire Data**: Systematically gather all data relevant to the research's **MAIN TASK**.
2.  **Plan Strategically**: Conduct iterative research, moving from general to specific topics.
3.  **Ensure Comprehensive Output**: Continuously analyze the **MAIN TASK** description to ensure all necessary data is collected.

---

# **WORKFLOW OVERVIEW**

## **Workflow nodes**
Each node has a specific role. Focus solely on the current node's task without anticipating subsequent steps.
- Task Specification: The user provides the main task description and all necessary details.
- Analysis: Assess the current knowledge base and formulate further research directions.
- Jobs Distribution: Based on the analysis, delegate specific research jobs to specialized agents.
- Jobs Execution: Agents execute the assigned jobs, gathering data and insights.
- 